In [ ]:
import tensorflow as tf

##### Encoder

In [ ]:
# input has to be a tensor, filter represents num of filters, 
# and k_size is for kernel size
def block(input, filter, k_size = (3, 3)):
  b = input
  for i in range(2):
    b = tf.keras.layers.Conv2D(filter = filter, 
                               kernel_size = k_size)(b)
    b = tf.keras.layers.Activation('relu')(b)
  return b

In [ ]:
# p_size is for max pooling and d_rate means dropout rate
def encoder_block(input, filter, p_size = (2, 2), d_rate = 0.3):
  f = block(input, filter = filter)
  p = tf.keras.layers.MaxPooling2D(p_size)(f)
  p = tf.keras.layers.Dropout(d_rate)(p)
  return f, p

In [ ]:
# Stack encoder_block to construct encoding part
def encoder(input):
  f1, p1 = encoder_block(input, 64)
  f2, p2 = encoder_block(p1, 128)
  f3, p3 = encoder_block(p2, 256)
  f4, p4 = encoder_block(p3, 512)
  return p4, (f1, f2, f3, f4)

###### Bottleneck

In [ ]:
def bottleneck(input):
  return block(input, filter = 1024)

###### Decoder

In [12]:
# e_output is an output from each layer of encoder
def decoder_block(input, e_output, filter, k_size = (3 , 3), 
                  strides = (2, 2), d_rate = 0.3):
  # h is for head
  h = tf.keras.layers.Conv2DTranpose(filter, k_size = k_size, 
                                     strides = strides, padding = 'same')(input)
  # b is for output block
  b = tf.keras.layers.concatenate([h, e_output])
  b = tf.keras.layers.Dropout(d_rate)(b)
  b = block(b, filter)
  return b

In [13]:
def decoder(input, e_output):
  f1, f2, f3, f4 = e_output
  c4 = decoder_block(input, f4, n_filters = 512)
  c3 = decoder_block(c4, f3, n_filters = 256)
  c2 = decoder_block(c3, f2, n_filters = 128)
  c1 = decoder_block(c2, f1, n_filters = 64)
  output = tf.keras.layers.COnv2D(3, (1, 1), activation = 'softmax')(c1)

###### Model

In [14]:
def UNET():
  input = tf.keras.layes.Input(shape = (128, 128, 3))
  encoder_output, conv = encoder(input)
  Bottleneck = bottleneck(encoder_output)
  output = decoder(Bottleneck, conv)
  unet = tf.keras.Model(inputs = input, outputs = output)
  return unet